In [1]:
import os
import csv
import json
import pickle
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from utils import Data

data_processor = Data(root_path="../")

for file in data_processor.data_dict.keys():
    print(file)

collection.sampled.tsv
train_sample_queries.tsv
train_sample_passv2_qrels.tsv
val_2021_53_queries.tsv
val_2021_passage_top100.txt
val_2021.qrels.pass.final.txt
test_2022_76_queries.tsv
test_2022_passage_top100.txt
test_2022.qrels.pass.withDupes.txt


In [2]:
for idx, row in enumerate(data_processor.yield_file('collection.sampled.tsv')):
    pid, passage = row
    doc = nlp(passage)
    print(f"Passage ID: {pid}")
    print(f"Passage content: {passage}")
    print([token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.is_space and token.is_alpha])
    break

读取文件的格式为('pid', 'passage')
Passage ID: msmarco_passage_05_512736984
Passage content: Discuss these Ribbit definitions with the community: × Close. Report Comment. We're doing our best to make sure our content is useful, accurate and safe. If by any chance you spot an inappropriate comment while navigating through our website please use this form to let us know, and we'll take care of it shortly.
['discuss', 'ribbit', 'definition', 'community', 'close', 'Report', 'Comment', 'good', 'sure', 'content', 'useful', 'accurate', 'safe', 'chance', 'spot', 'inappropriate', 'comment', 'navigate', 'website', 'use', 'form', 'let', 'know', 'care', 'shortly']


In [3]:
for idx, row in enumerate(data_processor.yield_file('train_sample_queries.tsv')):
    print(row)
    qid, query = row
    if idx > 2:
        break

读取文件的格式为('qid', 'query')
train_sample_queries(qid='989639', query='where is ogdensburg ny')
train_sample_queries(qid='661863', query='what foods should a person in diverticulitis exacerbation avoid')
train_sample_queries(qid='406306', query='is chris cornell married')
train_sample_queries(qid='132755', query='definition of basketball')


In [4]:
for idx, row in enumerate(data_processor.yield_file('train_sample_passv2_qrels.tsv')):
    qid, _, pid, rating = row
    print(row)
    if idx > 5:
        break

读取文件的格式为('qid', 'mark', 'pid', 'rating')
train_sample_passv2_qrels(qid='1185869', mark='0', pid='msmarco_passage_08_840101254', rating='1')
train_sample_passv2_qrels(qid='645590', mark='0', pid='msmarco_passage_61_310962365', rating='1')
train_sample_passv2_qrels(qid='827277', mark='0', pid='msmarco_passage_36_735439741', rating='1')
train_sample_passv2_qrels(qid='14562', mark='0', pid='msmarco_passage_00_867054103', rating='1')
train_sample_passv2_qrels(qid='708236', mark='0', pid='msmarco_passage_03_576438477', rating='1')
train_sample_passv2_qrels(qid='306992', mark='0', pid='msmarco_passage_03_757919813', rating='1')
train_sample_passv2_qrels(qid='441269', mark='0', pid='msmarco_passage_03_158685688', rating='1')


In [5]:
for idx, row in enumerate(data_processor.yield_file('val_2021_53_queries.tsv')):
    print(row)
    qid, query = row
    if idx > 2:
        break

读取文件的格式为('qid', 'query')
val_2021_53_queries(qid='2082', query='At about what age do adults normally begin to lose bone mass?')
val_2021_53_queries(qid='540006', query='was friedrich nietzsche an atheist')
val_2021_53_queries(qid='23287', query='are landlords liable if someone breaks in a hurts tenant')
val_2021_53_queries(qid='300025', query='how many whales are caught in fishing nets off california')


In [6]:
for idx, row in enumerate(data_processor.yield_file('val_2021_passage_top100.txt')):
    print(row)
    qid, _, pid, rank, score, sys_id = row
    if idx > 2:
        break

读取文件的格式为('qid', 'mark', 'pid', 'rank', 'score', 'sys_id')
val_2021_passage_top100(qid='2082', mark='Q0', pid='msmarco_passage_45_623131157', rank='1', score='19.820700', sys_id='Anserini')
val_2021_passage_top100(qid='2082', mark='Q0', pid='msmarco_passage_10_669481249', rank='2', score='18.954300', sys_id='Anserini')
val_2021_passage_top100(qid='2082', mark='Q0', pid='msmarco_passage_10_669572296', rank='3', score='18.954298', sys_id='Anserini')
val_2021_passage_top100(qid='2082', mark='Q0', pid='msmarco_passage_66_702392512', rank='4', score='18.954298', sys_id='Anserini')


In [7]:
for idx, row in enumerate(data_processor.yield_file('val_2021.qrels.pass.final.txt')):
    print(row)
    qid, _, pid, rating = row
    if idx > 2:
        break

读取文件的格式为('qid', 'mark', 'pid', 'rating')
val_2021_qrels_pass_final(qid='2082', mark='0', pid='msmarco_passage_01_552803451', rating='0')
val_2021_qrels_pass_final(qid='2082', mark='0', pid='msmarco_passage_01_790431544', rating='0')
val_2021_qrels_pass_final(qid='2082', mark='0', pid='msmarco_passage_02_179207466', rating='3')
val_2021_qrels_pass_final(qid='2082', mark='0', pid='msmarco_passage_02_179210433', rating='1')


In [8]:
for idx, row in enumerate(data_processor.yield_file('test_2022_76_queries.tsv')):
    print(row)
    qid, query = row
    if idx > 2:
        break

读取文件的格式为('qid', 'query')
test_2022_76_queries(qid='2000511', query='average bahamas temperature at the end of october')
test_2022_76_queries(qid='2000719', query='business architect role definition')
test_2022_76_queries(qid='2001532', query='example of what a family advocate does')
test_2022_76_queries(qid='2001908', query='how do I replace the burners on a ducane grill')


In [9]:
for idx, row in enumerate(data_processor.yield_file('test_2022_passage_top100.txt')):
    print(row)
    qid, _, pid, rank, score, sys_id = row
    if idx > 2:
        break

读取文件的格式为('qid', 'mark', 'pid', 'rank', 'score', 'sys_id')
test_2022_passage_top100(qid='2000511', mark='Q0', pid='msmarco_passage_05_149863652', rank='1', score='17.336300', sys_id='Anserini')
test_2022_passage_top100(qid='2000511', mark='Q0', pid='msmarco_passage_00_491588004', rank='2', score='16.496099', sys_id='Anserini')
test_2022_passage_top100(qid='2000511', mark='Q0', pid='msmarco_passage_00_491587144', rank='3', score='16.268999', sys_id='Anserini')
test_2022_passage_top100(qid='2000511', mark='Q0', pid='msmarco_passage_10_615818416', rank='4', score='15.435800', sys_id='Anserini')


In [10]:
for idx, row in enumerate(data_processor.yield_file('test_2022.qrels.pass.withDupes.txt')):
    print(row)
    qid, _, pid, rating = row
    if idx > 2:
        break

读取文件的格式为('qid', 'mark', 'pid', 'rating')
test_2022_qrels_pass_withDupes(qid='2000511', mark='0', pid='msmarco_passage_00_491550793', rating='0')
test_2022_qrels_pass_withDupes(qid='2000511', mark='0', pid='msmarco_passage_00_491572639', rating='0')
test_2022_qrels_pass_withDupes(qid='2000511', mark='0', pid='msmarco_passage_00_491574814', rating='0')
test_2022_qrels_pass_withDupes(qid='2000511', mark='0', pid='msmarco_passage_00_491575627', rating='0')


In [11]:
doc = nlp('The quick brown fox jumps over the lazy dog.')
displacy.render(doc, style='dep', jupyter=True)

In [12]:
# with doc.retokenize() as retokenizer:
#     retokenizer.merge(doc[1:3], attrs={"LEMMA": "quick_brown_fox"})
# displacy.render(doc, style='dep', jupyter=True)

In [13]:
# 生成依存树

class Node:
    def __init__(self, text, parent, children, dep):
        self.text = text
        self.parent = parent
        self.children = children
        self.dep = dep

nodes = {}
for token in doc:
    node = Node(
        text=token.text,
        parent=None,
        children=set(),
        dep=token.dep_
    )
    nodes[token.text] = node

for token in doc:
    text, head, dep, head_dep = token.text, token.head.text, token.dep_, token.head.dep_
    nodes[text].parent = nodes[head]
    nodes[head].children.add(nodes[text])

for text, node in nodes.items():
    if node.text == node.parent.text:
        root = node
        break

root.text

'jumps'